In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [2]:
tain_path = './Dataset/Train/'
test_path = './Dataset/Test/'
IMAGE_SIZE = [224,224]

In [3]:
mobilenet = VGG19(input_shape=IMAGE_SIZE+[3], weights='imagenet', include_top=False)

In [4]:
for layer in mobilenet.layers:
    layer.trainable = False

In [5]:
folder = glob('./Dataset/Train/*')
folder

['./Dataset/Train/Uninfected', './Dataset/Train/Parasite']

In [6]:
X = Flatten()(mobilenet.output)

In [7]:
prediction = Dense(len(folder), activation='softmax')(X)
model = Model(inputs = mobilenet.input, outputs = prediction)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [8]:
training_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2,
                                 horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
training_set = training_datagen.flow_from_directory(directory='./Dataset/Train/',
                                                   target_size=(224,224))

Found 416 images belonging to 2 classes.


In [10]:
test_set = test_datagen.flow_from_directory(directory='./Dataset/Test/',
                                                   target_size=(224,224))

Found 134 images belonging to 2 classes.


In [11]:
# model=Sequential()
# model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(224,224,3)))
# model.add(MaxPooling2D(pool_size=2))
# model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation ="relu"))
# model.add(MaxPooling2D(pool_size=2))
# model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
# model.add(MaxPooling2D(pool_size=2))
# model.add(Flatten())
# model.add(Dense(500,activation="relu"))
# model.add(Dense(2,activation="softmax"))
# model.summary()

In [12]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [14]:
result = model.fit_generator(generator=training_set, steps_per_epoch=len(training_set),
                            validation_data=test_set, epochs=7, validation_steps=len(test_set))

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/7
13/13 [==============================] - 182s 14s/step - loss: 1.3691 - accuracy: 0.5553 - val_loss: 1.0545 - val_accuracy: 0.6791
Epoch 2/7
13/13 [==============================] - 180s 14s/step - loss: 0.8007 - accuracy: 0.6466 - val_loss: 0.6222 - val_accuracy: 0.6716
Epoch 3/7
13/13 [==============================] - 179s 14s/step - loss: 0.4460 - accuracy: 0.7788 - val_loss: 0.4707 - val_accuracy: 0.7537
Epoch 4/7
13/13 [==============================] - 179s 14s/step - loss: 0.3608 - accuracy: 0.8462 - val_loss: 0.4045 - val_accuracy: 0.7537
Epoch 5/7
13/13 [==============================] - 179s 14s/step - loss: 0.3103 - accuracy: 0.8870 - val_loss: 0.4094 - val_accuracy: 0.7910
Epoch 6/7
13/13 [==============================] - 181s 14s/step - loss: 0.2840 - accuracy: 0.8822 - val_loss: 0.3690 - val_accuracy: 0.8582
Epoch 7/7
13/13 [==============================] - 179s 14s/step - loss: 0.275

In [15]:
from tensorflow.keras.models import load_model
model.save('final_model.h5')